# Get an Inventory of all MSSQL Parent Objects

In Rubrik, MSSQL Objects have a parent child relationship. The database is always the child, while the SQL Server Instance or the Availability Group is the parent. 

When databases first get put onto an Instance, the Instance is the parent of the child. When a DBA decides that a database should be inside an Availability Group, then the Availability Group becomes the parent. This is important information to know and understand. In Rubrik, the parent is also known as the **location** of the database. 

Think about a situation where you want to search Rubrik for a database name called AdventureWorks. Simply searching for the database name could return back multiple entries as the database could exist on multiple locations. To narrow down the number of returned records, you want to include a location. Either the Availability Group, or the SQL Server Instance. This ensure that you get back the correct occurance of the AdventureWorks database.

## Connect to Rubrik Security Cloud

In [1]:
Import-Module RubrikSecurityCloud
Connect-Rsc


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20230802-35     v20230802-41



## List all Parent Objects
For SQL Server objects, Rubrik works with a parent/child relationship. The database is always the child, but the parent can be different depending upon configuration. In the majority of use cases, the parent of a database is the SQL Server Instance. Databases in an Availability Group configuration, the parent is the Availability Group, not the SQL Server Instance. Even though the database may phsyically reside upon 2 or more instances of SQL Server, the parent is the AG. This is because, you can assign seperate SLA Domain to each replica server. In this case, Rubrik would have a difficult time performing conflict resolution if database in the AG could 2 or more SLA Domain rules goverening it depending upon backup preferences. To simplify this and not have conflicts, Rubrik makes the AG the parent of the database. Then, regardless of which replica server the backup was taken on, the same SLA manages the life cycle of the backup. 

In [2]:
$RSCMSSQLInventory = (Invoke-RscQueryMssql -TopLevelDescendant).Nodes
$RSCMSSQLInventory 


AuthorizedOperations        : 
ObjectType                  : WINDOWS_CLUSTER
SlaAssignment               : UNASSIGNED
ConfiguredSlaDomain         : 
EffectiveRetentionSlaDomain : 
EffectiveSlaDomain          : RubrikSecurityCloud.Types.GlobalSlaReply
PendingSla                  : 
ReplicatedObjects           : 
Id                          : 09610691-4cfb-541a-923e-4c0ade1b86c3
Name                        : RP-SQL19AGD-1
NumWorkloadDescendants      : 0
ReplicatedObjectCount       : 
SlaPauseStatus              : False
AllOrgs                     : 
Cluster                     : 
DescendantConnection        : 
EffectiveSlaSourceObject    : 
Hosts                       : 
LatestUserNote              : 
LogicalChildConnection      : 
LogicalPath                 : 
PendingObjectDeletionStatus : 
PhysicalPath                : 
PrimaryClusterLocation      : 
SnapshotDistribution        : 

AuthorizedOperations        : 
ObjectType                  : PHYSICAL_HOST
OsType                      

The data returned back is a simple list of Windows Clusters, Phsyical Hosts and Availability Groups. Additionally, when working with the SQL Server Instance, most instances are the defualt instance of MSSQLSERVER, but in some environments, they may have named instances. Like machine.domain.com\sql01 or machine.domain.com\sqlweb. From here we need to start to filter out records we do not want. 

## Get an Instance of SQL Server
### Using Filter Object
Because we are using GraqhQL, we have the ability to send an object that contains the fields we want to filter on. In the example below, we are creating a hashtable called `$filter` which contains our filter predicates. 

In [18]:
$filter = @(
    @{
        "field" = "NAME"
        "texts" = "rp-sql19sl-01.perf.rubrik.com"
    }
)
$inputs = Invoke-RscQueryMssql -TopLevelDescendant -GetInputs
$inputs.Arg.filter = $Filter
$RSCMSSQLInventory = Invoke-RscQueryMssql -TopLevelDescendant -Arg $inputs.Arg
$RSCMSSQLInventory.Nodes
$RSCMSSQLInventory.Nodes.PhysicalChildConnection.Nodes 


AuthorizedOperations        : 
ObjectType                  : PHYSICAL_HOST
OsType                      : 
SlaAssignment               : UNASSIGNED
ConfiguredSlaDomain         : 
EffectiveRetentionSlaDomain : 
EffectiveSlaDomain          : RubrikSecurityCloud.Types.GlobalSlaReply
PendingSla                  : 
ReplicatedObjects           : 
AgentId                     : 
CbtStatus                   : Enabled
CdmId                       : 
CdmLink                     : 
DefaultCbt                  : 
Id                          : d2f7bd73-0b1b-5912-8c7d-bac1c46d6217
IpAddresses                 : 
IsArchived                  : False
IsChangelistEnabled         : False
IsOracleHost                : True
Name                        : rp-sql19sl-01.perf.rubrik.com
NasMigrationInfo            : 
NasVendorType               : 
NumWorkloadDescendants      : 13
OsName                      : 
ReplicatedObjectCount       : 
SlaPauseStatus              : False
VfdState                    : NotInst

### Using Powershell Where-Object
This approach is more familiar and traditional to Powershell users. In this example, we are still filtering on host to get an SQL Server Instance. We are also adding an additional filter for the name of the SQL Server Instance. 

In [20]:
$SQLServerHost = "rp-sql19sl-01.perf.rubrik.com"
$RSCMSSQLInventory = (Invoke-RscQueryMssql -TopLevelDescendant).Nodes | Where-Object {$_.Name -eq $SQLServerHost}
$RSCMSSQLInventory
$RSCMSSQLInventory.PhysicalChildConnection.Nodes | Where-Object {$_.ObjectType -eq "MSSQL_INSTANCE" -and $_.Name -eq "MSSQLSERVER"}


AuthorizedOperations        : 
ObjectType                  : PHYSICAL_HOST
OsType                      : 
SlaAssignment               : UNASSIGNED
ConfiguredSlaDomain         : 
EffectiveRetentionSlaDomain : 
EffectiveSlaDomain          : RubrikSecurityCloud.Types.GlobalSlaReply
PendingSla                  : 
ReplicatedObjects           : 
AgentId                     : 
CbtStatus                   : Enabled
CdmId                       : 
CdmLink                     : 
DefaultCbt                  : 
Id                          : d2f7bd73-0b1b-5912-8c7d-bac1c46d6217
IpAddresses                 : 
IsArchived                  : False
IsChangelistEnabled         : False
IsOracleHost                : True
Name                        : rp-sql19sl-01.perf.rubrik.com
NasMigrationInfo            : 
NasVendorType               : 
NumWorkloadDescendants      : 13
OsName                      : 
ReplicatedObjectCount       : 
SlaPauseStatus              : False
VfdState                    : NotInst

With the above information, we get back some information. The important information is the **id** and **name** fields. These fields are used later to get databases or perform operations like Live Mount, Export, Restore, or take a backup. The data returned has a couple of different IDs. The id attached to the object type of PHYSICAL_HOST is not the ID you will need later for working SQL Server objects or operations. You will need the id found in the PhysicalChildConnection section and filter on object type of MSSQL_INSTANCE. If you are dealing with named instances, then you will want to add an additional predicate of name being equal to the name of the instance as shown in the exmaple above. 

## Get an Availability Group

In [9]:
$filter = @(
    @{
        "field" = "NAME"
        "texts" = "RR-SQL22-AG1"
    },
    @{
        "field" = "IS_ARCHIVED"
        "texts" = "false"
    },
    @{
        "field" = "IS_RELIC"
        "texts" = "false"
    },
    @{
        "field" = "IS_GHOST"
        "texts" = "false"
    }
    
)
$inputs = Invoke-RscQueryMssql -TopLevelDescendant -GetInputs
$inputs.Arg.filter = $Filter
$RSCMSSQLInventory = Invoke-RscQueryMssql -TopLevelDescendant -Arg $inputs.Arg
$RSCMSSQLInventory.Nodes
# $RSCMSSQLInventory.Nodes.PhysicalChildConnection.Nodes 

# $AGName = "RR-SQL22-AG1"
# $RSCMSSQLInventory = (Invoke-RscQueryMssql -TopLevelDescendant).Nodes | Where-Object {$_.ObjectType -eq "MSSQL_AVAILABILITY_GROUP" -and $_.Name -eq $AGName}
# $RSCMSSQLInventory


AuthorizedOperations        : 
ObjectType                  : MSSQL_AVAILABILITY_GROUP
SlaAssignment               : DIRECT
ConfiguredSlaDomain         : 
EffectiveRetentionSlaDomain : 
EffectiveSlaDomain          : RubrikSecurityCloud.Types.GlobalSlaReply
PendingSla                  : 
ReplicatedObjects           : 
CopyOnly                    : 
HasLogConfigFromSla         : True
HostLogRetention            : 
Id                          : daae59eb-66d8-58b2-b8ca-e525a7b47aa8
LogBackupFrequencyInSeconds : 
LogBackupRetentionInHours   : 
Name                        : RR-SQL22-AG1
NumWorkloadDescendants      : 1
ReplicatedObjectCount       : 
SlaPauseStatus              : False
AllOrgs                     : 
Cluster                     : 
DescendantConnection        : 
EffectiveSlaSourceObject    : 
Instances                   : 
LatestUserNote              : 
LogicalChildConnection      : 
LogicalPath                 : 
PendingObjectDeletionStatus : 
PhysicalPath                : 
Pri

In this example, we get back the **id** and **name** of the Availability Group. This will be information we will need going forward to work with SQL Server objects or do operational tasks. 